In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
from data_loader import load_names_from_web
import altair as alt

In [2]:
df = load_names_from_web(category='national', hide_pre_1937=True, use_existing_files=True)

In [3]:
df = df.sort_values(by='year')
df['cumsum'] = df.groupby(['name', 'M/F'])['count'].cumsum()
df['sum'] = df.groupby(['name', 'M/F'])['count'].transform('sum')

In [4]:
medians = df[df['cumsum'] >= df['sum']/2]
medians = medians.drop_duplicates(subset=['name', 'M/F'], keep='first')
medians['age'] = 2022 - medians['year']

In [5]:
medians[(medians['name'].isin(['Paul','Jonathan','Jordan'])) & (medians['M/F'] == 'M')]

,state,year,name,M/F,count,cumsum,sum,age
953181,US,1962,Paul,M,23809,551169,1072423,60
1134733,US,1990,Jonathan,M,24079,444232,855606,32
705199,US,1998,Jordan,M,14409,198254,389056,24


In [6]:
medians[medians['name'] == 'Karen']

,state,year,name,M/F,count,cumsum,sum,age
838586,US,1959,Karen,F,36774,512498,981868,63
792738,US,1964,Karen,M,91,1464,2759,58


In [7]:
thisyear = df[df['year'] == 2022][['name', 'M/F', 'count']].rename(columns={'count': 'thisyear_count'})
df2 = medians.merge(thisyear, how='left', on=['name', 'M/F'])
df2['thisyear_count'] = df2['thisyear_count'].fillna(0)

In [8]:
df3 = df2[df2['sum'] > 10000]
chart = alt.Chart(df3).mark_circle().encode(
    x=alt.X('age:Q', title='Age'),
    y=alt.Y('thisyear_count:Q', title='Count'),
    size='sum',
    tooltip=['name', 'M/F', 'age', 'thisyear_count', 'sum'],
    color=alt.condition(alt.datum['M/F'] == 'F', alt.value('#FF69B4'), alt.value('#85C1E9'))
).properties(
    width=450,
    height=400,
    title='Relationship between Age and Count of Names'
)


chart

alt.Chart(...)